In [1]:
from sx_multi.data_source import DataSource
from sx_multi.analysis import Analysis, analysis
from sx_multi.local_executor import LocalExecutor

from sx_multi.funcx_executor import FuncXExecutor



from servicex import ServiceXDataset
from sx_multi import FuncAdlQastle
from coffea import hist, processor


In [2]:
dids = ['mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00']
datasets = [
    ServiceXDataset(did, backend_type='xaod')
    for did in dids
]

In [3]:
ds = FuncAdlQastle()
leptons_per_event_query = ds \
        .Select(lambda e: e.Electrons("Electrons")) \
        .Select(lambda eles: eles.Where(lambda e: e.pt()/1000.0 > 30.0)) \
        .Select(lambda eles: eles.Where(lambda e: abs(e.eta()) < 2.5)) \
        .Where(lambda eles: len(eles) == 2) \
        .Select(lambda ls: (ls.Select(lambda e: e.pt()/1000.0), ls.Select(lambda e: e.eta()), ls.Select(lambda e: e.phi()), ls.Select(lambda e: e.m()/1000.0))) \
        .AsROOTTTree('data.root', 'mytree', ('ElePt', 'EleEta', 'ElePhi', 'EleM'))

leptons_per_event_query

In [4]:
datasource = DataSource(query=leptons_per_event_query, metadata={}, datasets=datasets)

In [5]:
class Z_EEAnalysis(Analysis):
    def __init__(self):
        self.accumulator = processor.dict_accumulator({
            "sumw": processor.defaultdict_accumulator(float),
            "mass": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("mass", "$Z_{ee}$ [GeV]", 60, 60, 120),
            ),
        })
        
    @staticmethod
    @analysis
    def process(events_url, tree_name, accumulator, events=None,):
        # Since we execute remotely, explicitly include everything we need.
        import awkward1 as ak
        from coffea.nanoevents import NanoEventsFactory
        from coffea.nanoevents import BaseSchema

        # This in is amazingly important - the invar mass will fail silently without it.
        # And must be done in here as this function is shipped off to the funcx processor
        # on a remote machine/remote python environment.
        from coffea.nanoevents.methods import candidate

        dataset = events.metadata['dataset']
        electrons = ak.zip({
            "pt": events.ElePt,
            "eta": events.EleEta,
            "phi": events.ElePhi,
            "mass": events.EleM,
            "charge": events.EleM,
        }, with_name="PtEtaPhiMCandidate")

        # The template of the output we will send back.
        output = accumulator.identity()

        # Do the buts, form the invar mass, plot.
        cut = (ak.num(electrons) == 2)
        diele = electrons[cut][:, 0] + electrons[cut][:, 1]

        output["sumw"][dataset] += len(events)
        output["mass"].fill(
            dataset=dataset,
            mass=diele.mass,
        )

        return output


In [6]:
analysis = Z_EEAnalysis()

executor = FuncXExecutor(endpoint_id='d0a328a1-320d-4221-9d53-142189f77e71')

async for hist in executor.execute(analysis, datasource):
    print(hist)




InvalidStateError: Result is not set.

unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
Cool rown
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
Cool rown
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent (<string>, line 1)
unexpected indent 